In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash"
)



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [33]:
import joblib
import pandas as pd

def predict_next_day_close(data):

    """
    This tool predicts the next day's India VIX value based on the provided inputs which are: open_price, high_price, low_price, prev_close.
    """
    open_price, high_price, low_price, prev_close = data['open_price'], data['high_price'], data['low_price'], data['prev_close']
    model = joblib.load('india_vix_forecast_model.pkl')
    scaler_X = joblib.load('scaler_X.pkl')
    scaler_y = joblib.load('scaler_y.pkl')

    overnight_change = (open_price - prev_close) / prev_close * 100

    new_df = pd.DataFrame([{
        'Open': open_price,
        'High': high_price,
        'Low': low_price,
        'Prev. Close': prev_close,
        'Overnight_NIFTY_Change': overnight_change
    }])

    new_scaled = scaler_X.transform(new_df)
    pred_scaled = model.predict(new_scaled)
    predicted_close = scaler_y.inverse_transform(pred_scaled.reshape(-1,1))

    return float(predicted_close[0][0])

def get_current_vix():
    """
    This tool helps to get the latest India VIX values such as: open_price, high_price, low_price, prev_close.
    """
    return {"open_price":15.1200, "high_price":15.5000, "low_price":14.9500, "prev_close":15.0250}

In [24]:
def interpret_vix_change(current_vix: float, predicted_vix: float) -> str:

    change_pct = ((predicted_vix - current_vix) / current_vix) * 100
    if change_pct > 2:
        interpretation = "Higher volatility expected."
    elif change_pct < -2:
        interpretation = "Lower volatility expected."
    else:
        interpretation = "Stable volatility outlook."
    return interpretation, round(change_pct, 2)

def forecast_and_interpret_vix(data):
    """
    This tool forecasts the next day's India VIX value and provides a brief interpretation 
    of the expected market volatility.

    The function internally:
    - Fetches the latest India VIX data (Open, High, Low, and Previous Close) using `get_current_vix()`.
    - Uses a trained ML model through `predict_vix_next_day()` to forecast the next day's VIX.
    - Interprets the predicted change using `interpret_vix_change()`:
        * If VIX ↑ → "Higher volatility expected."
        * If VIX ↓ → "Lower volatility expected."
        * If change is small → "Stable volatility outlook."

    Returns:
        str: A concise financial summary including:
             - Input VIX data
             - Predicted next-day VIX
             - Percentage change
             - Short interpretation of expected volatility
             
    Example output:
        "Based on today's India VIX data (Open: 15.12, High: 15.50, Low: 14.95, Prev Close: 15.03), 
         the forecast for tomorrow's VIX is 14.21 (-1.3%). Interpretation: Lower volatility expected."
    """
    predicted_vix = predict_next_day_close(data)
    interpretation, change_pct = interpret_vix_change(data["prev_close"], predicted_vix)

    response = (
        f"Based on today's India VIX data (Open: {data['open_price']}, High: {data['high_price']}, "
        f"Low: {data['low_price']}, Prev Close: {data['prev_close']}), "
        f"the forecast for tomorrow's VIX is {predicted_vix:.2f} ({change_pct:+.2f}%). "
        f"Interpretation: {interpretation}"
    )
    return response


In [34]:
from langgraph.prebuilt import create_react_agent

system_prompt = """
You are a financial market assistant providing analysis and insights.
Use the provided model to forecast volatility and give a brief interpretation:
- If VIX ↑ → say "Higher volatility expected."
- If VIX ↓ → say "Lower volatility expected."
- If change small → say "Stable volatility outlook."
Keep responses short and clear.
You can also answer general financial questions like:
"What does a rising VIX mean?" or "How does NIFTY affect volatility?"
"""


tools = [forecast_and_interpret_vix, get_current_vix]

agent = create_react_agent(
    llm,
    tools
)


In [35]:
agent.invoke(
    {"messages": [{"role":"system", "content": system_prompt},{"role": "user", "content": "What’s the expected VIX tomorrow?"}]}
)

{'messages': [SystemMessage(content='\nYou are a financial market assistant providing analysis and insights.\nUse the provided model to forecast volatility and give a brief interpretation:\n- If VIX ↑ → say "Higher volatility expected."\n- If VIX ↓ → say "Lower volatility expected."\n- If change small → say "Stable volatility outlook."\nKeep responses short and clear.\nYou can also answer general financial questions like:\n"What does a rising VIX mean?" or "How does NIFTY affect volatility?"\n', additional_kwargs={}, response_metadata={}, id='cc3cedee-0771-4cb4-8edb-043a940a9286'),
  HumanMessage(content='What’s the expected VIX tomorrow?', additional_kwargs={}, response_metadata={}, id='a67e371b-cbfb-4b4f-8802-ffc9245b6de0'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'forecast_and_interpret_vix', 'arguments': '{}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'sa

In [ ]:
import pandas as pd
import joblib
from datetime import datetime, timedelta


def get_current_vix(date_range: str = "latest"):
    """
    This tool fetches India VIX data from a CSV file for the desired timeframe.

    It can handle different query scenarios, such as:
    - Getting the most recent day's VIX data (default)
    - Fetching 1 week or N days of data
    - Filtering between two specific dates (if mentioned)

    Parameters:
        date_range (str, optional): Timeframe for which VIX data is required.
            Examples:
                "latest" → returns the latest day's data
                "7d" or "1w" → returns data for the past 7 days
                "2025-10-15 to 2025-10-22" → returns data between those dates

    Returns:
        dict:
            If single-day data:
                {
                    "open_price": float,
                    "high_price": float,
                    "low_price": float,
                    "prev_close": float
                }
            If multiple days:
                {
                    "data": DataFrame converted to list of dicts
                }
    """
    
    # Load your dataset
    df = pd.read_csv("sample_dataset_23-25.csv")  # Ensure this file path is correct
    df["Date "] = pd.to_datetime(df["Date "])
    df = df.sort_values("Date ")

    # Handle 'latest' case → return the most recent day's record
    if date_range.lower() == "latest":
        latest = df.iloc[-1]
        return {
            "open_price": float(latest["Open "]),
            "high_price": float(latest["High "]),
            "low_price": float(latest["Low "]),
            "prev_close": float(latest["Prev. Close "])
        }

    # Handle N-day or week queries (e.g., "7d", "1w", "5 days")
    if "d" in date_range or "w" in date_range:
        # Convert to number of days
        if "w" in date_range:
            days = int(date_range.replace("w", "")) * 7
        else:
            days = int(date_range.replace("d", ""))
        
        end_date = df["Date "].max()
        start_date = end_date - timedelta(days=days)
        filtered = df[(df["Date "] >= start_date) & (df["Date "] <= end_date)]
        
        return {
            "data": filtered.to_dict(orient="records")
        }

    # Handle explicit date range queries: "YYYY-MM-DD to YYYY-MM-DD"
    if "to" in date_range:
        start_str, end_str = [x.strip() for x in date_range.split("to")]
        start_date = datetime.strptime(start_str, "%Y-%m-%d")
        end_date = datetime.strptime(end_str, "%Y-%m-%d")

        filtered = df[(df["Date "] >= start_date) & (df["Date "] <= end_date)]
        return {
            "data": filtered.to_dict(orient="records")
        }

    # Default fallback
    raise ValueError("Invalid date_range format. Use 'latest', '7d', or 'YYYY-MM-DD to YYYY-MM-DD'.")


In [40]:
from langgraph.prebuilt import create_react_agent

system_prompt = """
You are a financial market assistant providing analysis and insights on India VIX and market volatility.

Capabilities:
1. You can forecast next-day VIX values using the trained ML model (`forecast_and_interpret_vix` tool).
2. You can fetch historical or recent VIX data from a CSV file (`get_current_vix` tool) for a given timeframe 
   such as "today", "this week", "last 7 days", or a specific date range.
3. When given a date range, analyze how the VIX has changed over that period and summarize the trend.

Guidelines:
- If VIX ↑ → say "Higher volatility expected."
- If VIX ↓ → say "Lower volatility expected."
- If change small → say "Stable volatility outlook."
- If asked about reasons or trends (e.g., "Why is VIX increasing this week?"), 
  use the historical data to describe the direction and percentage change in VIX.

Keep responses short, data-driven, and easy to understand.

You can also answer general financial questions such as:
- "What does a rising VIX mean?"
- "How does NIFTY affect volatility?"
- "Why is market volatility important?"
"""


tools = [forecast_and_interpret_vix, get_current_vix]

agent = create_react_agent(
    llm,
    tools
)

In [44]:
agent.invoke(
    {"messages": [{"role":"system", "content": system_prompt},{"role": "user", "content": "Why is VIX increasing this week?"}]}
)

/var/folders/4q/3fv4pvb53j96tnvsqpcqx2c40000gn/T/ipykernel_2682/427686243.py:65: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Date "] = pd.to_datetime(df["Date "])
/var/folders/4q/3fv4pvb53j96tnvsqpcqx2c40000gn/T/ipykernel_2682/427686243.py:65: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Date "] = pd.to_datetime(df["Date "])


{'messages': [SystemMessage(content='\nYou are a financial market assistant providing analysis and insights on India VIX and market volatility.\n\nCapabilities:\n1. You can forecast next-day VIX values using the trained ML model (`forecast_and_interpret_vix` tool).\n2. You can fetch historical or recent VIX data from a CSV file (`get_current_vix` tool) for a given timeframe \n   such as "today", "this week", "last 7 days", or a specific date range.\n3. When given a date range, analyze how the VIX has changed over that period and summarize the trend.\n\nGuidelines:\n- If VIX ↑ → say "Higher volatility expected."\n- If VIX ↓ → say "Lower volatility expected."\n- If change small → say "Stable volatility outlook."\n- If asked about reasons or trends (e.g., "Why is VIX increasing this week?"), \n  use the historical data to describe the direction and percentage change in VIX.\n\nKeep responses short, data-driven, and easy to understand.\n\nYou can also answer general financial questions suc